In [2]:
import csv
import pandas as pd
import numpy as np

In [59]:
import pandas as pd
import glob

# Set the pattern to match your 10 CSV files
# Example: "*.csv" for all CSV files in the directory
csv_files_pattern = "/Users/huiyu/Documents/uchicago/2024spring/spring-30123/anomaly-dataset/*.csv"

# Read all the CSV files matching the pattern
csv_files = glob.glob(csv_files_pattern)

# List to hold DataFrames
dataframes = []

# Loop through each file and read them into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Now, combined_df contains the data from all 10 files
# You can write it back to a new CSV file if needed:
combined_df.to_csv("/Users/huiyu/Documents/uchicago/2024spring/spring-30123/anomaly-dataset/combined_csv.csv", index=False)


In [94]:
df_anomaly = pd.read_csv("/Users/huiyu/Documents/uchicago/2024spring/spring-30123/anomaly-dataset/combined_csv.csv")

In [95]:
df_af = pd.read_csv("af_mines.csv")

In [96]:
df_af.head()

,FID,vid,id,source,project,pcode,name,visit_date,visit,novisit_reason,...,actor3name_digging,actor3name_monopoly,roadblocks,roadblocks_actor,environment_water_change,environment_deforestation,conflict_category,conflict_type,accident_injured,accident_dead
0,caf_mines_curated_all_opendata_p_ipis.fid--355...,1,15,IPIS,IPIS - TetraTech 2019,cafmine00015,Pouadoungbe,3/28/19,1,NaN,...,NaN,NaN,0.0,NaN,1.0,extension chauffage,NaN,NaN,0.0,0.0
1,caf_mines_curated_all_opendata_p_ipis.fid--355...,2,1,IPIS,IPIS - TetraTech 2019,cafmine00001,Adouzekede,4/22/19,1,NaN,...,NaN,NaN,0.0,NaN,1.0,etayement extension chauffage,Conflit lié _ l__xploitation artisanale,"Conflits de limites, Litige de propriété entre...",1.0,0.0
2,caf_mines_curated_all_opendata_p_ipis.fid--355...,3,16,IPIS,IPIS - TetraTech 2019,cafmine00016,Nkongue,4/22/19,1,NaN,...,NaN,NaN,0.0,NaN,1.0,extension,NaN,NaN,0.0,0.0
3,caf_mines_curated_all_opendata_p_ipis.fid--355...,4,2,IPIS,IPIS - TetraTech 2019,cafmine00002,Adouzekede,4/23/19,1,NaN,...,NaN,NaN,0.0,NaN,1.0,etayement extension chauffage,NaN,NaN,1.0,0.0
4,caf_mines_curated_all_opendata_p_ipis.fid--355...,5,3,IPIS,IPIS - TetraTech 2019,cafmine00003,Ngom-Mbata,4/23/19,1,NaN,...,NaN,NaN,0.0,NaN,1.0,etayement extension chauffage,"Conflit lié _ l__xploitation artisanale, Confl...","Conflits de limites, Litige de propriété du si...",0.0,0.0


In [97]:
df_af = df_af.rename(columns={
    'longitude': 'Longitude',
    'latitude': 'Latitude'
})

In [98]:
df_af[['Longitude','Latitude']]

,Longitude,Latitude
0,17.838062,3.830327
1,17.467914,4.794698
2,17.837117,3.833920
3,17.464800,4.796248
4,17.502983,4.760425
...,...,...
909,16.125267,3.561943
910,16.236954,3.403430
911,16.245850,3.401320
912,16.248991,3.257713


In [99]:
data1 = df_af[['Longitude','Latitude']]
data2 = df_anomaly

In [100]:
combined = pd.merge(data1, data2, on=['Longitude', 'Latitude'], how='outer', indicator=True)

In [67]:
combined

,Longitude,Latitude,_merge
0,17.838062,3.830327,left_only
1,17.838062,3.830327,left_only
2,17.838062,3.830327,left_only
3,17.467914,4.794698,left_only
4,17.467914,4.794698,left_only
...,...,...,...
5622,17.675835,6.003037,right_only
5623,17.682752,6.004923,right_only
5624,17.676464,6.011840,right_only
5625,17.673320,6.012469,right_only


In [68]:
combined['Match'] = combined['_merge'].apply(lambda x: 1 if x == 'both' else 0)

In [69]:
combined.to_csv("marked_match.csv")

In [72]:
combined.describe()

,Longitude,Latitude,Match
count,5627.000000,5627.000000,5627.0
mean,15.799892,4.504460,0.0
std,0.966854,0.916387,0.0
min,14.634638,3.219773,0.0
25%,14.882928,4.025396,0.0
50%,15.769880,4.244854,0.0
75%,16.668779,4.795517,0.0
max,17.893852,6.947788,0.0


### 1.2 diameter match

In [103]:
longitude_threshold = 0.05
latitude_threshold = 0.05

In [101]:
df_anomaly

,Longitude,Latitude
0,16.655888,3.910950
1,16.653373,3.912837
2,16.657146,3.912837
3,16.652744,3.913466
4,16.653373,3.913466
...,...,...
4708,17.675835,6.003037
4709,17.682752,6.004923
4710,17.676464,6.011840
4711,17.673320,6.012469


In [102]:
data2

,Longitude,Latitude
0,16.655888,3.910950
1,16.653373,3.912837
2,16.657146,3.912837
3,16.652744,3.913466
4,16.653373,3.913466
...,...,...
4708,17.675835,6.003037
4709,17.682752,6.004923
4710,17.676464,6.011840
4711,17.673320,6.012469


In [104]:
def mark_nearby(data1, data2, lon_thresh, lat_thresh):
    def is_near(row):
        # Check if any point in data1 is within the specified thresholds of the point in data2
        nearby = data1[
            (np.abs(data1['Longitude'] - row['Longitude']) <= lon_thresh) &
            (np.abs(data1['Latitude'] - row['Latitude']) <= lat_thresh)
        ]
        # Return 1 if any points match, otherwise 0
        return 1 if not nearby.empty else 0

    # Apply `is_near` to each row of dataset2
    data2['Marked'] = data2.apply(is_near, axis=1)
    return data2


In [105]:
# Process Dataset 2 using Dataset 1 as a reference
data2_marked = mark_points(data1, data2, longitude_threshold, latitude_threshold)

# View the result
print(data2_marked)
data2_marked.to_csv("combined_1.csv")

      Longitude  Latitude  Marked
0     16.655888  3.910950       0
1     16.653373  3.912837       0
2     16.657146  3.912837       0
3     16.652744  3.913466       0
4     16.653373  3.913466       0
...         ...       ...     ...
4708  17.675835  6.003037       0
4709  17.682752  6.004923       0
4710  17.676464  6.011840       0
4711  17.673320  6.012469       0
4712  17.673949  6.012469       0

[4713 rows x 3 columns]


In [106]:
data2_marked.describe()

,Longitude,Latitude,Marked
count,4713.000000,4713.000000,4713.000000
mean,15.718729,4.346300,0.093359
std,0.950642,0.833115,0.290965
min,14.634859,3.219876,0.000000
25%,14.770055,3.983894,0.000000
50%,15.641600,4.128522,0.000000
75%,16.544587,4.354898,0.000000
max,17.821093,6.928032,1.000000


# part2 AWS

In [16]:
from pystac_client import Client

LandsatSTAC = Client.open("https://landsatlook.usgs.gov/stac-server", headers=[])

for collection in LandsatSTAC.get_collections():
    print(collection)


<CollectionClient id=landsat-c2l2-sr>
<CollectionClient id=landsat-c2l2-st>
<CollectionClient id=landsat-c2ard-st>
<CollectionClient id=landsat-c2l2alb-bt>
<CollectionClient id=landsat-c2l3-fsca>
<CollectionClient id=landsat-c2ard-bt>
<CollectionClient id=landsat-c2l1>
<CollectionClient id=landsat-c2l3-ba>
<CollectionClient id=landsat-c2l2alb-st>
<CollectionClient id=landsat-c2ard-sr>
<CollectionClient id=landsat-c2l2alb-sr>
<CollectionClient id=landsat-c2l2alb-ta>
<CollectionClient id=landsat-c2l3-dswe>
<CollectionClient id=landsat-c2ard-ta>


In [26]:
import boto3
import json

In [27]:
sqs = boto3.client('sqs')
sns = boto3.client('sns')
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
dynamodb = boto3.resource('dynamodb')
dynamo_client = boto3.client('dynamodb')
kinesis = boto3.client('kinesis')

ClientError: An error occurred (ExpiredToken) when calling the GetRole operation: The security token included in the request is expired

In [17]:
def BuildSquare(lon, lat, delta):
    c1 = [lon + delta, lat + delta]
    c2 = [lon + delta, lat - delta]
    c3 = [lon - delta, lat - delta]
    c4 = [lon - delta, lat + delta]
    geometry = {"type": "Polygon", "coordinates": [[ c1, c2, c3, c4, c1 ]]}
    return geometry

geometry = BuildSquare(-59.346271, -34.233076, 0.04)

In [18]:
timeRange = '2019-06-01/2021-06-01'

In [19]:
LandsatSearch = LandsatSTAC.search ( 
    intersects = geometry,
    datetime = timeRange,
    query =  ['eo:cloud_cover95'],
    collections = ["landsat-c2l2-sr"] )

Landsat_items = [i.to_dict() for i in LandsatSearch.get_items()]
print(f"{len(Landsat_items)} Landsat scenes fetched")

/Users/huiyu/opt/anaconda3/lib/python3.9/site-packages/pystac_client/item_search.py:834: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


193 Landsat scenes fetched


In [21]:
for item in Landsat_items:
    red_href = item['assets']['red']['href']
    red_s3 = item['assets']['red']['alternate']['s3']['href']
    print(red_href)    
    print(red_s3)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_SR_B4.TIF
s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210527_20210622_02_T1/LE07_L2SP_226084_20210527_20210622_02_T1_SR_B3.TIF
s3://usgs-landsat/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210527_20210622_02_T1/LE07_L2SP_226084_20210527_20210622_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/etm/2021/225/084/LE07_L2SP_225084_20210520_20210615_02_T1/LE07_L2SP_225084_20210520_20210615_02_T1_SR_B3.TIF
s3://usgs-landsat/collection02/level-2/standard/etm/2021/225/084/LE07_L2SP_225084_20210520_20210615_02_T1/LE07_L2SP_225084_20210520_20210615_02_T1_SR_B3.TIF


In [29]:
Landsat_items[0]['assets']['red']

{'href': 'https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_SR_B4.TIF',
 'type': 'image/vnd.stac.geotiff; cloud-optimized=true',
 'title': 'Red Band (B4)',
 'description': 'Collection 2 Level-2 Red Band (B4) Surface Reflectance',
 'eo:bands': [{'name': 'B4',
   'common_name': 'red',
   'gsd': 30,
   'center_wavelength': 0.65}],
 'alternate': {'s3': {'storage:platform': 'AWS',
   'storage:requester_pays': True,
   'href': 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_SR_B4.TIF'}},
 'file:checksum': '1340504d9692fea8ac14ae73b56a01c5c4f3447e1a98ef45b5c68e6bbbb345e40ee5975a1c5df56a4d914b53248218dc07b152886d273241fe6737abc0d8915035b4',
 'roles': ['data']}

In [25]:
def plotNDVI(nir,red,filename):
    ndvi = (nir-red)/(nir+red)
    ndvi[ndvi>0.15] = 1

In [ ]:
from pyproj import Transformer

def getSubset(geotiff_file, bbox):
    with rio.Env(aws_session):
        with rio.open(geotiff_file) as geo_fp:
            # Calculate pixels with PyProj 
            Transf = Transformer.from_crs("epsg:4326", geo_fp.crs) 
            lat_north, lon_west = Transf.transform(bbox[3], bbox[0])
            lat_south, lon_east = Transf.transform(bbox[1], bbox[2]) 
            x_top, y_top = geo_fp.index( lat_north, lon_west )
            x_bottom, y_bottom = geo_fp.index( lat_south, lon_east )
            # Define window in RasterIO
            window = rio.windows.Window.from_slices( ( x_top, x_bottom ), ( y_top, y_bottom ) )                
            # Actual HTTP range request
            subset = geo_fp.read(1, window=window)
    return subset

In [ ]:
for i,item in enumerate(Landsat_items):
    red_s3 = item['assets']['red']['alternate']['s3']['href']
    nir_s3 = item['assets']['nir08']['alternate']['s3']['href']
    date = item['properties']['datetime'][0:10]
    print("Landsat item number " + str(i) + "/" + str(len(Landsat_items)) + " " + date)
    red = getSubset(red_s3, bbox)
    nir = getSubset(nir_s3, bbox)
    plotNDVI(nir,red,"landsat/" + date + "_ndvi.png")